In [2]:
from bs4 import BeautifulSoup
import json
import requests
import polars as pl
from shared.utils import formalize_data
import datetime

In [3]:
df_cw_gn = pl.read_parquet("./coin_walk_db_gn.parquet")
df_cw_kbl = pl.read_parquet("./coin_walk_db_kbl.parquet")
df_cw_tnk = pl.read_parquet("./coin_walk_db_tnk.parquet")

print(f'{len(df_cw_gn)=} | cluster_size={len(df_cw_gn)/1024}')
print(f'{len(df_cw_kbl)=} | cluster_size={len(df_cw_kbl)/1024}')
print(f'{len(df_cw_tnk)=} | cluster_size={len(df_cw_tnk)/1024}')

len(df_cw_gn)=324632 | cluster_size=317.0234375
len(df_cw_kbl)=3838380 | cluster_size=3748.41796875
len(df_cw_tnk)=15890700 | cluster_size=15518.26171875


In [4]:
df_cw_gn[0]


cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,bolillasid
str,str,str,str,str,str,str,str,str,str,str
"""A""","""A1""","""A11""","""A111""","""A1111""","""A11111""","""A111111""","""A1111111""","""A11111111""","""A111111111""","""0102030405"""


In [5]:

url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')



In [6]:
lst = []
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  cols[0] = (datetime.datetime.strptime(cols[0], "%d/%m/%Y").strftime("%Y-%m-%d"))
  id, bolillas_int = formalize_data(bolillas_str=cols[2])
  cols.append(id)
  
  lst.append(cols)

df_gn = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'bolillasid'
  ],
  orient='row'
)

TOTAL_LINES = len(df_gn)
print(f'{TOTAL_LINES=}')
display(df_gn.head(10))

TOTAL_LINES=7718


fecha,sorteo,bolillas,bolillasid
str,str,str,str
"""2025-04-02""","""4174""","""26 14 24 11 17""","""1114172426"""
"""2025-04-01""","""4173""","""12 25 33 13 08""","""0812132533"""
"""2025-03-31""","""4172""","""07 22 34 26 16""","""0716222634"""
"""2025-03-30""","""4171""","""35 05 15 29 03""","""0305152935"""
"""2025-03-29""","""4170""","""23 06 04 09 10""","""0406091023"""
"""2025-03-28""","""4169""","""20 10 01 09 29""","""0109102029"""
"""2025-03-27""","""4168""","""27 35 05 32 08""","""0508273235"""
"""2025-03-26""","""4167""","""34 29 06 24 26""","""0624262934"""
"""2025-03-25""","""4166""","""29 06 35 22 04""","""0406222935"""


In [7]:

df_clustered = df_gn.join(df_cw_gn, on="bolillasid")

In [8]:

df_clustered = df_clustered.sort('fecha', descending=True)
df_clustered.write_excel("./coin_walk_gn.xlsx")
df_clustered[0:10]

fecha,sorteo,bolillas,bolillasid,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2025-04-02""","""4174""","""26 14 24 11 17""","""1114172426""","""A""","""A2""","""A22""","""A221""","""A2211""","""A22112""","""A221122""","""A2211222""","""A22112222""","""A221122222"""
"""2025-04-01""","""4173""","""12 25 33 13 08""","""0812132533""","""B""","""B2""","""B22""","""B222""","""B2222""","""B22222""","""B222222""","""B2222221""","""B22222211""","""B222222112"""
"""2025-03-31""","""4172""","""07 22 34 26 16""","""0716222634""","""A""","""A2""","""A22""","""A222""","""A2222""","""A22221""","""A222211""","""A2222112""","""A22221121""","""A222211211"""
"""2025-03-30""","""4171""","""35 05 15 29 03""","""0305152935""","""B""","""B1""","""B12""","""B122""","""B1222""","""B12222""","""B122221""","""B1222211""","""B12222112""","""B122221122"""
"""2025-03-29""","""4170""","""23 06 04 09 10""","""0406091023""","""B""","""B1""","""B11""","""B111""","""B1111""","""B11112""","""B111122""","""B1111222""","""B11112221""","""B111122211"""
"""2025-03-28""","""4169""","""20 10 01 09 29""","""0109102029""","""A""","""A1""","""A12""","""A121""","""A1212""","""A12121""","""A121212""","""A1212121""","""A12121212""","""A121212122"""
"""2025-03-27""","""4168""","""27 35 05 32 08""","""0508273235""","""A""","""A2""","""A21""","""A212""","""A2121""","""A21212""","""A212121""","""A2121211""","""A21212112""","""A212121121"""
"""2025-03-26""","""4167""","""34 29 06 24 26""","""0624262934""","""A""","""A2""","""A21""","""A212""","""A2121""","""A21211""","""A212111""","""A2121111""","""A21211112""","""A212111122"""
"""2025-03-25""","""4166""","""29 06 35 22 04""","""0406222935""","""A""","""A2""","""A21""","""A211""","""A2112""","""A21122""","""A211221""","""A2112212""","""A21122122""","""A211221221"""


In [9]:
def show_b_stats (col: str, df: pl.DataFrame):
  df_viz = df.group_by(
    col
  ).agg(pl.count(col).alias('count')).sort('count', descending=True)

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)

  return df_viz

In [10]:
show_b_stats('cluster_1', df_clustered)

cluster_1,count
str,u32
"""A""",3877
"""B""",3841


alt.Chart(...)

cluster_1,count
str,u32
"""A""",3877
"""B""",3841


In [11]:
show_b_stats('cluster_2', df_clustered)

cluster_2,count
str,u32
"""A2""",1943
"""B2""",1938
"""A1""",1934
"""B1""",1903


alt.Chart(...)

cluster_2,count
str,u32
"""A2""",1943
"""B2""",1938
"""A1""",1934
"""B1""",1903


In [12]:
show_b_stats('cluster_3', df_clustered)

cluster_3,count
str,u32
"""A11""",1011
"""B21""",1000
"""A22""",978
"""B11""",976
"""A21""",965
"""B22""",938
"""B12""",927
"""A12""",923


alt.Chart(...)

cluster_3,count
str,u32
"""A11""",1011
"""B21""",1000
"""A22""",978
"""B11""",976
"""A21""",965
"""B22""",938
"""B12""",927
"""A12""",923


In [13]:
show_b_stats('cluster_4', df_clustered)

cluster_4,count
str,u32
"""A212""",525
"""A111""",515
"""A222""",503
"""B212""",502
"""B111""",500
…,…
"""A121""",464
"""A122""",459
"""B222""",455


alt.Chart(...)

cluster_4,count
str,u32
"""A212""",525
"""A111""",515
"""A222""",503
"""B212""",502
"""B111""",500
…,…
"""A121""",464
"""A122""",459
"""B222""",455


In [14]:
show_b_stats('cluster_5', df_clustered)

cluster_5,count
str,u32
"""B2122""",265
"""A2121""",264
"""A1121""",261
"""A2122""",261
"""A1112""",261
…,…
"""B1212""",223
"""B2221""",222
"""B1211""",222


alt.Chart(...)

cluster_5,count
str,u32
"""B2122""",265
"""A2121""",264
"""A1121""",261
"""A2122""",261
"""A1112""",261
…,…
"""B1212""",223
"""B2221""",222
"""B1211""",222


In [15]:
show_b_stats('cluster_6', df_clustered)

cluster_6,count
str,u32
"""A11122""",153
"""A21221""",139
"""B12211""",138
"""B21221""",137
"""A21211""",136
…,…
"""A21121""",102
"""B12121""",101
"""A21112""",98


alt.Chart(...)

cluster_6,count
str,u32
"""A11122""",153
"""A21221""",139
"""B12211""",138
"""B21221""",137
"""A21211""",136
…,…
"""A21121""",102
"""B12121""",101
"""A21112""",98


In [16]:
show_b_stats('cluster_7', df_clustered)

cluster_7,count
str,u32
"""A112122""",84
"""A111221""",82
"""B111112""",78
"""A222112""",76
"""B112112""",76
…,…
"""B112111""",45
"""B122122""",44
"""A211122""",43


alt.Chart(...)

cluster_7,count
str,u32
"""A112122""",84
"""A111221""",82
"""B111112""",78
"""A222112""",76
"""B112112""",76
…,…
"""B112111""",45
"""B122122""",44
"""A211122""",43


In [17]:
show_b_stats('cluster_8', df_clustered)

cluster_8,count
str,u32
"""A1112212""",47
"""B1111211""",44
"""A1121222""",44
"""A2221122""",44
"""B1112111""",44
…,…
"""A1212221""",18
"""A1122221""",17
"""B2212112""",16


alt.Chart(...)

cluster_8,count
str,u32
"""A1112212""",47
"""B1111211""",44
"""A1121222""",44
"""A2221122""",44
"""B1112111""",44
…,…
"""A1212221""",18
"""A1122221""",17
"""B2212112""",16


In [18]:
show_b_stats('cluster_9', df_clustered)

cluster_9,count
str,u32
"""B22122111""",26
"""B11112111""",25
"""A11122122""",25
"""A21211111""",24
"""B11121112""",24
…,…
"""A12122121""",7
"""A11221121""",6
"""B11112211""",6


alt.Chart(...)

cluster_9,count
str,u32
"""B22122111""",26
"""B11112111""",25
"""A11122122""",25
"""A21211111""",24
"""B11121112""",24
…,…
"""A12122121""",7
"""A11221121""",6
"""B11112211""",6


In [19]:
test = show_b_stats('cluster_10', df_clustered)
test.write_excel("./gn_cluster10.xlsx")

cluster_10,count
str,u32
"""A221222121""",16
"""A111221222""",16
"""B211221211""",15
"""B221221111""",15
"""A221211221""",15
…,…
"""B221111222""",2
"""B212121212""",2
"""B111122221""",1


alt.Chart(...)